In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
import tweepy as tw #python twitter api
import sys
import csv
import re #regular expression
import networkx #Network of words analysis
import itertools
import collections
import matplotlib.pyplot as plt #plots
import nltk #library for sentiment analysis
from nltk.corpus import stopwords
from nltk import bigrams
import networkx as nx
import textblob #open source library for sent analysis
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [3]:
data=pd.read_csv('fitbitdata.csv')

In [4]:
data.head()

,id,created_at,text,favoritecount,retweetcount
0,1300151721415462913,2020-08-30 19:21:40,@mr_brejoc Hi. We appreciate your interest and...,0,0
1,1300051885638873088,2020-08-30 12:44:57,"@ollyvale Hi, Olly. Sorry to hear about this. ...",0,0
2,1299958125747822593,2020-08-30 06:32:23,"@KittyL96 Hey, @KittyL96. We're constantly str...",0,0
3,1299767444131573761,2020-08-29 17:54:41,"@redranger36 Hey, @redranger36. Glad to hear y...",0,0
4,1299686284907094016,2020-08-29 12:32:11,@Haituga 1/2 Unser Team für Produktentwicklung...,0,0


In [5]:
dataworeplies=data[~data.text.str.get(0).isin(['@','.'])]
dataworeplies.head()

,id,created_at,text,favoritecount,retweetcount
8,1299456992872820736,2020-08-28 21:21:04,RT @WebMD: WebMD Chief Medical Officer @drjohn...,0,5
9,1299443470260744192,2020-08-28 20:27:20,"Stress affects us all, but why exactly do we f...",25,8
10,1299410712276725763,2020-08-28 18:17:10,Camo. Rainbow. Coral. Charcoal. All made with ...,42,8
22,1298736934450106368,2020-08-26 21:39:49,And the award for best press reaction goes to....,108,12
24,1298666150830542848,2020-08-26 16:58:33,✅ Pace\n✅ Distance\n✅ Heart Rate\n❌ Phone\n\nL...,60,16


In [6]:
def remove_url(txt):
    """Replace URLs found in a text string with nothing 
    (i.e. it will remove the URL from the string).

    Parameters
    ----------
    txt : string
        A text string that you want to parse and remove urls.

    Returns
    -------
    The same txt string with url's removed.
    """

    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

In [7]:
from nltk.corpus import stopwords
import string
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['RT', '@','fitbit']

In [8]:
dataworeplies['textclean'] = dataworeplies['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
dataworeplies['textclean'] = [remove_url(tweet) for tweet in dataworeplies['textclean']]

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
dataworeplies.head()

,id,created_at,text,favoritecount,retweetcount,textclean
8,1299456992872820736,2020-08-28 21:21:04,RT @WebMD: WebMD Chief Medical Officer @drjohn...,0,5,WebMD WebMD Chief Medical Officer drjohnwhyte ...
9,1299443470260744192,2020-08-28 20:27:20,"Stress affects us all, but why exactly do we f...",25,8,Stress affects us all exactly feel causes it F...
10,1299410712276725763,2020-08-28 18:17:10,Camo. Rainbow. Coral. Charcoal. All made with ...,42,8,Camo Rainbow Coral Charcoal All made REPREVE r...
22,1298736934450106368,2020-08-26 21:39:49,And the award for best press reaction goes to....,108,12,And award best press reaction goes to
24,1298666150830542848,2020-08-26 16:58:33,✅ Pace\n✅ Distance\n✅ Heart Rate\n❌ Phone\n\nL...,60,16,Pace Distance Heart Rate Phone Leave phone sti...


In [11]:
textjson=dataworeplies['textclean']
jsondata = textjson.to_json() 
print(jsondata)

{"8":"WebMD WebMD Chief Medical Officer drjohnwhyte speaks Dr John Moore Medical Director fitbit independent study","9":"Stress affects us all exactly feel causes it Fitbit research scientist Dr Samy Abdel","10":"Camo Rainbow Coral Charcoal All made REPREVE recycled fibers These new accessories Sense amp Versa 3 l","22":"And award best press reaction goes to","24":"Pace Distance Heart Rate Phone Leave phone still see stats builtin GPS new","29":"FastCompany Why Fitbit calls newest wearable health smartwatch","30":"CNETNews How Fitbit Sense triples new sensors COVID19 era","31":"businessinsider Fitbit launched new smartwatch tell stressed are beating Apple punch","37":"Innovation knows you Meet FitbitSense first health smartwatch helps manage stress monit","38":"A L L N E W L I N E U P A L E R T","44":"Have lost 100lbs using Fitbit device We want hear you Tell us weight loss jou","47":"After feeling like missing life Lindy decided time change Read Lindy uses Fitbit t","48":"Since start 

In [12]:
import json

# parse x:
tweetsjson = json.loads(jsondata)

tweetsjson.values()

dict_values(['WebMD WebMD Chief Medical Officer drjohnwhyte speaks Dr John Moore Medical Director fitbit independent study', 'Stress affects us all exactly feel causes it Fitbit research scientist Dr Samy Abdel', 'Camo Rainbow Coral Charcoal All made REPREVE recycled fibers These new accessories Sense amp Versa 3 l', 'And award best press reaction goes to', 'Pace Distance Heart Rate Phone Leave phone still see stats builtin GPS new', 'FastCompany Why Fitbit calls newest wearable health smartwatch', 'CNETNews How Fitbit Sense triples new sensors COVID19 era', 'businessinsider Fitbit launched new smartwatch tell stressed are beating Apple punch', 'Innovation knows you Meet FitbitSense first health smartwatch helps manage stress monit', 'A L L N E W L I N E U P A L E R T', 'Have lost 100lbs using Fitbit device We want hear you Tell us weight loss jou', 'After feeling like missing life Lindy decided time change Read Lindy uses Fitbit t', 'Since start global health crisis Fitbit research te

### use this one for sentiment analysis since this has the json values [text] and not the keys which might impact the sentiment score